In [150]:
# Import Packages

import pandas as pd
import numpy as np
import os
import pickle

from sklearn.model_selection import train_test_split
from tcn import TCN
from tensorflow.keras.layers import Input, Normalization, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, GRU
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential

In [151]:
with open('all_teams_data.pkl', 'rb') as file:
    team_data = pickle.load(file)

print("Data loaded from 'all_teams_data.pkl'")

Data loaded from 'all_teams_data.pkl'


In [152]:
team_data.shape

(266, 163, 51)

In [153]:
playoff_teams = pd.read_csv('playoff_teams.csv')
success = playoff_teams['Level of Success']
success = success.to_numpy()

In [154]:
categories, inverse = np.unique(success, return_inverse=True)
y_all = np.zeros((success.size, categories.size))
y_all[np.arange(success.size), inverse] = 1

In [155]:
y_all.shape

(266, 5)

In [156]:
X_train, X_test, y_train, y_test = train_test_split(team_data, y_all, test_size = 0.2)
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

In [157]:
norm_layer = Normalization()
norm_layer.adapt(X_train)

In [163]:
team_model = Sequential()
team_model.add(Input(shape=(team_data.shape[1], team_data.shape[2])))
# team_model.add(TCN(nb_filters=10, dilations=[1, 2, 4, 8, 16, 32], dropout_rate=0.2, use_skip_connections=True, use_batch_norm=True))
# team_model.add(LSTM(256, return_sequences=True))
for i in range(15):
    team_model.add(Conv1D(128, 3, activation='relu'))
team_model.add(MaxPooling1D(2))
# team_model.add(LSTM(256, return_sequences=True))
team_model.add(Conv1D(256, 3, activation='relu'))
team_model.add(MaxPooling1D(2))
# team_model.add(LSTM(256, return_sequences=True))
team_model.add(Conv1D(64, 3, activation='relu'))
team_model.add(LSTM(256))
team_model.add(Flatten())
team_model.add(Dense(128, activation='relu'))
team_model.add(Dense(y_all.shape[1]))
team_model.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_58 (Conv1D)              │ (None, 161, 128)       │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_59 (Conv1D)              │ (None, 159, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_60 (Conv1D)              │ (None, 157, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_61 (Conv1D)              │ (None, 155, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_62 (Conv1D)              │ (None, 153, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_63 (Conv1D)              │ (None, 151, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_64 (Conv1D)              │ (None, 149, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_65 (Conv1D)              │ (None, 147, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_66 (Conv1D)              │ (None, 145, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_67 (Conv1D)              │ (None, 143, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_68 (Conv1D)              │ (None, 141, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_69 (Conv1D)              │ (None, 139, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_70 (Conv1D)              │ (None, 137, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_71 (Conv1D)              │ (None, 135, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_72 (Conv1D)              │ (None, 133, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_35 (MaxPooling1D) │ (None, 66, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_73 (Conv1D)              │ (None, 64, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_36 (MaxPooling1D) │ (None, 32, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_74 (Conv1D)              │ (None, 30, 64)         │        49,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 256)            │       328,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,653 (4.65 MB)

 Trainable params: 1,219,653 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [165]:
optimizer = Adam(learning_rate=0.0000001, clipvalue=1.0)

team_model.compile(optimizer=optimizer,
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [167]:
team_model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 253ms/step - accuracy: 0.2255 - loss: 5.4862
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 264ms/step - accuracy: 0.1996 - loss: 5.7905
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 251ms/step - accuracy: 0.1972 - loss: 5.8880
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 249ms/step - accuracy: 0.2385 - loss: 5.3822
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.2216 - loss: 5.4126
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 255ms/step - accuracy: 0.2205 - loss: 5.0842
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 255ms/step - accuracy: 0.2288 - loss: 5.2122
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 278ms/step - accuracy: 0.1933 - loss: 5.3916
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 265ms/step - accuracy: 0.2354 - loss: 5.6603
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 254ms/step - accuracy: 0.2320 - loss: 5.3937


In [168]:
team_model.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.2438 - loss: 5.0935


[5.37317419052124, 0.24074074625968933]